# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [21]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket
os.environ['SERPAPI_API_KEY'] = 'YOUR SERPAPI_API_KEY'
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'

import openai

openai.api_key = os.environ['OPENAI_API_KEY']

In [22]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [23]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [24]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True,handle_parsing_errors=True
)

In [25]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
Could not parse output:  No. 
Final answer: 第28届大运会 (The 28th Universiade)
Intermediate answer: Invalid or incomplete response
Could not parse output: 
Intermediate answer: Invalid or incomplete response
So the final answer is: 第28届大运会 (The 28th Universiade)

> Finished chain.


'第28届大运会 (The 28th Universiade)'

In [26]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "非典肺炎的爆发地在那里"
)



> Entering new AgentExecutor chain...
Could not parse output:  No.
Intermediate answer: Invalid or incomplete response
So the final answer is: Invalid or incomplete response

> Finished chain.


'Invalid or incomplete response'

In [27]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))

d:\Sdk\Anaconda3\envs\homework\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




成都举办的大运会是第31届大运会。


In [28]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [9]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [10]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [11]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the English translation of "2023年大运会"?
Intermediate answer: No good search result found
The phrase "2023年大运会" translates to "2023 Universiade" in English. The Universiade is an international multi-sport event, organized for university athletes by the International University Sports Federation (FISU).

Follow up: Where is the 2023 Universiade being held?
Intermediate answer: Lake Placid 2023 FISU World University Games Winter, 12-22 January. Welcome back to legendary Lake Placid, where many Universiade and Olympic Legends were born.
So the final answer is: Lake Placid, New York, United States

> Finished chain.


'Lake Placid, New York, United States'